In [1]:
import requests
from bs4 import BeautifulSoup as bs
import lxml
import pandas as pd

In [211]:
#fuctions for verifying


def is_a_conceptial(string): #проверяем, есть ли у фразеологизма пояснение к значению
    return len(re.findall('\(.*?\)$', string)) > 0


def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))


def has_latin(text):
    return bool(re.search('[a-zA-Z]', text))


def check_phrase(phrase): # определяет, идет ли фраза на сайт или остается для приложения
        
        if not has_cyrillic(phrase): # если работаем с идиомой
            lengths_of_words = [len(word) for word in phrase.split()]
            s = 0
            last_index = -1
            for i in range(len(lengths_of_words)):
                if s + lengths_of_words[i] < 16:
                    s += lengths_of_words[i] + 1
                    last_index = i
                else:
                    break
            if (sum(lengths_of_words[last_index + 1:]) + len(lengths_of_words[last_index + 1:]) - 1) < 16:
                return True
            else:
                return False
        
        else: # если работаем со значением
            if len(re.findall(' \(.*?\)$', phrase)):
                return False
            else:
                if len(phrase) > 30:
                    return False
                else:
                    return True
                

def add_checking_columns(df):
    draft_df = df
    draft_df['check_phrase'] = draft_df['фраза'].apply(lambda phrase: check_phrase(phrase)) # если True, то идет в основную таблицу
    draft_df['check_transl'] = draft_df['перевод'].apply(lambda transl: check_phrase(transl)) # если True, то идет в основную таблицу
    return draft_df


def for_the_app(draft_df):
    return draft_df[draft_df['check_phrase'] & draft_df['check_transl']]


def for_the_web(draft_df):
    return draft_df[(draft_df['check_phrase'] == False) | (draft_df['check_transl'] == False)]


def create_the_app_df(df):
    draft_df = add_checking_columns(df)
    return for_the_app(draft_df)


def create_the_web_df(df):
    draft_df = add_checking_columns(df)
    return for_the_web(draft_df)

In [98]:
#function for parsing


def parse_the_pages(url, total_num_of_pages):
    for i in range(1, total_num_of_pages + 1):
        if i == 1:
            #parsing a page without a number
            r = requests.get(url)
            html = bs(r.content, 'html.parser')
            items = html.select(".pane__body > .pane__text")
            meanings = []
            for el in items:
                meanings.append(el.text)


            #meanings
            '''парсим идиомы'''


            page = requests.get(url)
            soup = bs(page.text, "html.parser")
            phrases = []


            for el in soup.find_all('span', itemprop="name"):
                    phrases.append(el.text)
            phrases = phrases[:len(meanings)]
        
        
        else:
            #parsing numbered pages
            new_url = url + '/' + str(i)
            page = requests.get(new_url)
            soup = bs(page.text, "html.parser")


            '''парсим идиомы'''
            draft_phrases = []

            for el in soup.find_all('span', itemprop="name"):
                draft_phrases.append(el.text)
            draft_phrases = draft_phrases[:35]
            phrases += draft_phrases


            '''парсим значения'''
            r = requests.get(new_url)
            html = bs(r.content, 'html.parser')


            items = html.select(".pane__body > .pane__text")
            draft_meanings = []
            for el in items:
                draft_meanings.append(el.text)
            meanings += draft_meanings


            phrases = phrases[:len(meanings)]


    return phrases, meanings

In [64]:
url = 'https://www.native-english.ru/idioms/category/body'


phrases, meanings = parse_first_page(url)


for el in list(zip(phrases, meanings)):
    print(el)

('able to breathe easily/freely again', 'снова вздохнуть свободно')
('all in one breath', 'на одном дыхании')
('as broad as a barn door', 'очень толстый, (широкий как амбарная дверь)')
('as dry as a bone', 'очень сухой, высохший; (сухой как кость)')
("as soft as a baby's bottom", 'очень мягкий и гладкий (как кожа младенца)')
("at each other's throats", 'драться, спорить всё время (брать друг друга за горло)')
("at the top of one's lungs", '(кричать) очень громко; во всё горло')
('back-to-back', '(сидеть) спина к спине')
('bad blood (between people)', 'вражда, неприязненные отношения между людьми')
('bare bones (of something)', 'самые основные части чего-либо')
("behind one's back", '(делать что-либо) за чьей-либо спиной')
('blood is thicker than water', 'свой своему поневоле брат; кровь не вода')
('blood runs cold', 'внушать или вселять ужас, кровь стынет в жилах')
('blood, sweat, and tears', 'огромное усилие (кровь, пот и слёзы)')
('blue blood', 'голубая кровь (аристократическое проис

In [65]:
phrases, meanings = parse_the_rest_pages(url, phrases, meanings, 5)


for el in list(zip(phrases, meanings)):
    print(el)


print(len(list(zip(phrases, meanings))))

('able to breathe easily/freely again', 'снова вздохнуть свободно')
('all in one breath', 'на одном дыхании')
('as broad as a barn door', 'очень толстый, (широкий как амбарная дверь)')
('as dry as a bone', 'очень сухой, высохший; (сухой как кость)')
("as soft as a baby's bottom", 'очень мягкий и гладкий (как кожа младенца)')
("at each other's throats", 'драться, спорить всё время (брать друг друга за горло)')
("at the top of one's lungs", '(кричать) очень громко; во всё горло')
('back-to-back', '(сидеть) спина к спине')
('bad blood (between people)', 'вражда, неприязненные отношения между людьми')
('bare bones (of something)', 'самые основные части чего-либо')
("behind one's back", '(делать что-либо) за чьей-либо спиной')
('blood is thicker than water', 'свой своему поневоле брат; кровь не вода')
('blood runs cold', 'внушать или вселять ужас, кровь стынет в жилах')
('blood, sweat, and tears', 'огромное усилие (кровь, пот и слёзы)')
('blue blood', 'голубая кровь (аристократическое проис

In [66]:
direct = ['en-ru']*len(list(zip(phrases, meanings)))
body_en_ru_idioms = {'направление': direct, 'фраза': phrases, 'перевод': meanings}
body_en_ru_idioms_df = pd.DataFrame(body_en_ru_idioms)


body_en_ru_idioms_df

,направление,фраза,перевод
0,en-ru,able to breathe easily/freely again,снова вздохнуть свободно
1,en-ru,all in one breath,на одном дыхании
2,en-ru,as broad as a barn door,"очень толстый, (широкий как амбарная дверь)"
3,en-ru,as dry as a bone,"очень сухой, высохший; (сухой как кость)"
4,en-ru,as soft as a baby's bottom,очень мягкий и гладкий (как кожа младенца)
...,...,...,...
164,en-ru,words stick in one's throat,"с трудом говорить из-за волнения, ""слова застр..."
165,en-ru,wring (someone's) neck,быть очень сердитым на кого-либо (желать сверн...
166,en-ru,yellow-bellied,чрезвычайно робкий или трусливый
167,en-ru,yoke around (someone's) neck,"ярмо на чьей-либо шее, обуза"


In [81]:
list_of_headings = ['Animals Birds Body Business Cards Clothes Color Education eyes-and-ears Food hand-and-arm head-and-mind Heart insects-and-fish Law leg-and-foot Medical Money mouth-and-teeth Negotiations Numbers Relationships Sports']
list_of_headings = [el.lower() for el in list_of_headings[0].split()]
list_of_headings

['animals',
 'birds',
 'body',
 'business',
 'cards',
 'clothes',
 'color',
 'education',
 'eyes-and-ears',
 'food',
 'hand-and-arm',
 'head-and-mind',
 'heart',
 'insects-and-fish',
 'law',
 'leg-and-foot',
 'medical',
 'money',
 'mouth-and-teeth',
 'negotiations',
 'numbers',
 'relationships',
 'sports']

In [91]:
numbers_of_pages = [5, 3, 5, 5, 1, 2, 3, 2, 4, 5, 5, 7, 2, 2, 7, 3, 1, 5, 2, 3, 4, 2, 3]
len(list_of_headings) == len(numbers_of_pages)

True

In [92]:
headings_numbers_dict = dict(zip(list_of_headings, numbers_of_pages))
headings_numbers_dict

{'animals': 5,
 'birds': 3,
 'body': 5,
 'business': 5,
 'cards': 1,
 'clothes': 2,
 'color': 3,
 'education': 2,
 'eyes-and-ears': 4,
 'food': 5,
 'hand-and-arm': 5,
 'head-and-mind': 7,
 'heart': 2,
 'insects-and-fish': 2,
 'law': 7,
 'leg-and-foot': 3,
 'medical': 1,
 'money': 5,
 'mouth-and-teeth': 2,
 'negotiations': 3,
 'numbers': 4,
 'relationships': 2,
 'sports': 3}

In [97]:
list(headings_numbers_dict.keys())

['animals',
 'birds',
 'body',
 'business',
 'cards',
 'clothes',
 'color',
 'education',
 'eyes-and-ears',
 'food',
 'hand-and-arm',
 'head-and-mind',
 'heart',
 'insects-and-fish',
 'law',
 'leg-and-foot',
 'medical',
 'money',
 'mouth-and-teeth',
 'negotiations',
 'numbers',
 'relationships',
 'sports']

## Creating tables for each category

In [100]:
import re


url = 'https://www.native-english.ru/idioms/category/'
tables_dict = {}


for head in list(headings_numbers_dict.keys()):
    new_url = url + head
    num_of_pages = headings_numbers_dict[head]
    phrases, meanings = parse_the_pages(new_url, num_of_pages)
    
    
    direct = ['en-ru']*len(list(zip(phrases, meanings)))
    if '-' in head:
        table_heading = re.sub('-', '_', head) + '_en_ru_idioms_df'
    else:
        table_heading = head + '_en_ru_idioms_df'
    
    
    idioms_dict = {'направление': direct, 'фраза': phrases, 'перевод': meanings}
    tables_dict[table_heading] = pd.DataFrame(idioms_dict)

    
tables_dict['body_en_ru_idioms_df']

,направление,фраза,перевод
0,en-ru,able to breathe easily/freely again,снова вздохнуть свободно
1,en-ru,all in one breath,на одном дыхании
2,en-ru,as broad as a barn door,"очень толстый, (широкий как амбарная дверь)"
3,en-ru,as dry as a bone,"очень сухой, высохший; (сухой как кость)"
4,en-ru,as soft as a baby's bottom,очень мягкий и гладкий (как кожа младенца)
...,...,...,...
164,en-ru,words stick in one's throat,"с трудом говорить из-за волнения, ""слова застр..."
165,en-ru,wring (someone's) neck,быть очень сердитым на кого-либо (желать сверн...
166,en-ru,yellow-bellied,чрезвычайно робкий или трусливый
167,en-ru,yoke around (someone's) neck,"ярмо на чьей-либо шее, обуза"


In [101]:
list(tables_dict.keys())

['animals_en_ru_idioms_df',
 'birds_en_ru_idioms_df',
 'body_en_ru_idioms_df',
 'business_en_ru_idioms_df',
 'cards_en_ru_idioms_df',
 'clothes_en_ru_idioms_df',
 'color_en_ru_idioms_df',
 'education_en_ru_idioms_df',
 'eyes_and_ears_en_ru_idioms_df',
 'food_en_ru_idioms_df',
 'hand_and_arm_en_ru_idioms_df',
 'head_and_mind_en_ru_idioms_df',
 'heart_en_ru_idioms_df',
 'insects_and_fish_en_ru_idioms_df',
 'law_en_ru_idioms_df',
 'leg_and_foot_en_ru_idioms_df',
 'medical_en_ru_idioms_df',
 'money_en_ru_idioms_df',
 'mouth_and_teeth_en_ru_idioms_df',
 'negotiations_en_ru_idioms_df',
 'numbers_en_ru_idioms_df',
 'relationships_en_ru_idioms_df',
 'sports_en_ru_idioms_df']

In [103]:
sum_of_lenghts = 0
for table_name in list(tables_dict.keys()):
    length = len(tables_dict[table_name])
    sum_of_lenghts += length
print(sum_of_lenghts)

2513


In [77]:
url = 'https://www.native-english.ru/idioms/category/body'
phrases, meanings = parse_the_pages(url, phrases, meanings, 5)


direct = ['en-ru']*len(list(zip(phrases, meanings)))
body_en_ru_idioms = {'направление': direct, 'фраза': phrases, 'перевод': meanings}
body_en_ru_idioms_df = pd.DataFrame(body_en_ru_idioms)


body_en_ru_idioms_df

,направление,фраза,перевод
0,en-ru,able to breathe easily/freely again,снова вздохнуть свободно
1,en-ru,all in one breath,на одном дыхании
2,en-ru,as broad as a barn door,"очень толстый, (широкий как амбарная дверь)"
3,en-ru,as dry as a bone,"очень сухой, высохший; (сухой как кость)"
4,en-ru,as soft as a baby's bottom,очень мягкий и гладкий (как кожа младенца)
...,...,...,...
164,en-ru,words stick in one's throat,"с трудом говорить из-за волнения, ""слова застр..."
165,en-ru,wring (someone's) neck,быть очень сердитым на кого-либо (желать сверн...
166,en-ru,yellow-bellied,чрезвычайно робкий или трусливый
167,en-ru,yoke around (someone's) neck,"ярмо на чьей-либо шее, обуза"


In [80]:
import re
s = 'head-and-mind'
s1 = re.sub('-', '_', s)
s1

'head_and_mind'

In [90]:
url = 'https://www.native-english.ru/idioms/category/medical'
phrases, meanings = parse_the_pages(url, phrases, meanings, 1)


#phrases = phrases[:len(meanings)]


direct = ['en-ru']*len(list(zip(phrases, meanings)))
medical_en_ru_idioms = {'направление': direct, 'фраза': phrases, 'перевод': meanings}
medical_en_ru_idioms_df = pd.DataFrame(medical_en_ru_idioms)


medical_en_ru_idioms_df

,направление,фраза,перевод
0,en-ru,at death's door,на пороге смерти
1,en-ru,break out in (something),"выступить, появиться (например, сыпь на теле)"
2,en-ru,bring (someone) around,"восстанавливать чьё-либо здоровье, приводить в..."
3,en-ru,bring (someone) to,приводить в чувство
4,en-ru,catch a cold,простудиться
5,en-ru,catch one's death of cold,очень сильно простудиться
6,en-ru,check-up,осмотр (врачом)
7,en-ru,clean bill of health,документ о (хорошем) состоянии здоровья челове...
8,en-ru,come down with (something),заболеть чем-либо
9,en-ru,couch doctor,психоаналитик


In [108]:
s = 'finger in every pie'
for w in s.split():
    print(len(w))

6
2
5
3


In [116]:
def check_phrase(phrase):
    if len(phrase) > 30:
        return False
    else:
        if phrase in all_idioms_df['фраза']:
            lengths_of_words = [len(word) for word in phrase.split()]
            s = 0
            last_index = -1
            for i in range(len(lengths_of_words)):
                if s + lengths_of_words[i] < 16:
                    s += lengths_of_words[i] + 1
                    last_index = i
                else:
                    break
            if (sum(lengths_of_words[last_index + 1:]) + len(lengths_of_words[last_index + 1:]) - 1) < 16:
                return True
            else:
                return False
        else:
            return True
            


In [118]:
check_phrase('fingers introduction every pie')

False

In [115]:
phrase = 'finger in every pie'
lengths_of_words = [len(word) for word in phrase.split()]
lengths_of_words

[6, 2, 5, 3]

In [119]:
tables_dict['cards_en_ru_idioms_df']

,направление,фраза,перевод
0,en-ru,above board,"честный (игроки показывали свою честность, дер..."
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас"
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег"
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...
5,en-ru,call a spade a spade,называть вещи своими именами
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую..."
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось"
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)"


In [146]:
transl = 'использовать лучшие ресурсы (козыри и просто карты)'
len(transl)

51

In [147]:
transl.split('(')

['использовать', 'лучшие', 'ресурсы', '(козыри', 'и', 'просто', 'карты)']

In [148]:
transl.split()[-1]

'карты)'

In [138]:
words = transl.split()
for tr in words:
    #print(tr[0] == '(', words.index(tr))
    if tr[0] == '(' and words.index(tr) > 0:
        words.remove(tr)

        
' '.join(words)

'использовать лучшие ресурсы'

In [145]:
#words.remove('(козыри)')
words[-1]

'ресурсы'

In [153]:
def is_a_conceptial(string):
    return len(re.findall('\(.*?\)$', string)) > 0

In [141]:
draft_df = tables_dict['cards_en_ru_idioms_df']
draft_df

,направление,фраза,перевод
0,en-ru,above board,"честный (игроки показывали свою честность, дер..."
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас"
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег"
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...
5,en-ru,call a spade a spade,называть вещи своими именами
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую..."
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось"
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)"


In [149]:
draft_df['фраза'][31], draft_df['перевод'][31]

('when the chips are down',
 '(быть) в трудной ситуации (chips – жетоны в казино)')

In [168]:
line =  draft_df['перевод'][31]
in_brakets = re.findall(' \(.*?\)$', line)
in_brakets[0][1:]
#opt = re.sub(r'[^\w\s]','', inp_str) 

'(chips – жетоны в казино)'

In [155]:
line

'(быть) в трудной ситуации (chips – жетоны в казино)'

In [164]:
s = '(Learn Python) (not C++)'
re.findall('\(.*?\)$', s)


['(Learn Python) (not C++)']

In [169]:
def get_the_concept(mean):
    in_brakets = re.findall(' \(.*?\)$', line)[0][1:]
    return in_brakets[in_brakets.find('(')+1:in_brakets.find(')')]

In [170]:
mean = draft_df['перевод'][31]
get_the_concept(mean)

'chips – жетоны в казино'

In [171]:
draft_df

,направление,фраза,перевод
0,en-ru,above board,"честный (игроки показывали свою честность, дер..."
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас"
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег"
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...
5,en-ru,call a spade a spade,называть вещи своими именами
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую..."
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось"
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)"


In [179]:
draft_df['check_phrase'] = draft_df['фраза'].apply(lambda phrase: check_phrase(phrase)) # если True, то идет в основную таблицу
draft_df['check_transl'] = draft_df['перевод'].apply(lambda transl: check_phrase(transl))
draft_df

,направление,фраза,перевод,check_phrase
0,en-ru,above board,"честный (игроки показывали свою честность, дер...",True
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас",True
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег",True
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...,False
5,en-ru,call a spade a spade,называть вещи своими именами,True
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую...",True
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...,True
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось",True
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)",True


In [182]:
draft_df = draft_df.drop(columns = ['check_phrase'],axis = 1)
draft_df

,направление,фраза,перевод
0,en-ru,above board,"честный (игроки показывали свою честность, дер..."
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас"
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег"
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...
5,en-ru,call a spade a spade,называть вещи своими именами
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую..."
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось"
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)"


In [183]:
draft_df['check_phrase'] = draft_df['фраза'].apply(lambda phrase: check_phrase(phrase)) # если True, то идет в основную таблицу
draft_df['check_transl'] = draft_df['перевод'].apply(lambda transl: check_phrase(transl)) # если True, то идет в основную таблицу
draft_df

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,above board,"честный (игроки показывали свою честность, дер...",True,False
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True,False
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас",True,False
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег",True,False
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...,None,False
5,en-ru,call a spade a spade,называть вещи своими именами,True,True
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую...",True,False
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...,True,False
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось",True,False
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)",True,False


In [187]:
check_phrase(draft_df['фраза'][4])

False

In [188]:
draft_df = draft_df.drop(columns = ['check_phrase', 'check_transl'],axis = 1)
draft_df

,направление,фраза,перевод
0,en-ru,above board,"честный (игроки показывали свою честность, дер..."
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас"
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег"
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...
5,en-ru,call a spade a spade,называть вещи своими именами
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую..."
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось"
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)"


In [189]:
draft_df['check_phrase'] = draft_df['фраза'].apply(lambda phrase: check_phrase(phrase)) # если True, то идет в основную таблицу
draft_df['check_transl'] = draft_df['перевод'].apply(lambda transl: check_phrase(transl)) # если True, то идет в основную таблицу
draft_df

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,above board,"честный (игроки показывали свою честность, дер...",True,False
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True,False
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас",True,False
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег",True,False
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...,False,False
5,en-ru,call a spade a spade,называть вещи своими именами,True,True
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую...",True,False
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...,True,False
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось",True,False
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)",True,False


In [192]:
draft_df[draft_df['check_phrase'] & draft_df['check_transl']]

,направление,фраза,перевод,check_phrase,check_transl
5,en-ru,call a spade a spade,называть вещи своими именами,True,True
13,en-ru,have an ace up one's sleeve,"иметь козырь ""про запас""",True,True
18,en-ru,in the cards,"вероятный, возможный",True,True
30,en-ru,up the ante,повысить ставки,True,True


In [195]:
def for_the_app(df):
    draft_df = df
    draft_df['check_phrase'] = draft_df['фраза'].apply(lambda phrase: check_phrase(phrase)) # если True, то идет в основную таблицу
    draft_df['check_transl'] = draft_df['перевод'].apply(lambda transl: check_phrase(transl)) # если True, то идет в основную таблицу

    
    return draft_df[draft_df['check_phrase'] & draft_df['check_transl']]


def for_the_web(df):
    draft_df = df
    draft_df['check_phrase'] = draft_df['фраза'].apply(lambda phrase: check_phrase(phrase)) # если True, то идет в основную таблицу
    draft_df['check_transl'] = draft_df['перевод'].apply(lambda transl: check_phrase(transl)) # если True, то идет в основную таблицу

    
    return draft_df[not draft_df['check_phrase'] or not draft_df['check_transl']]

In [196]:
draft_df

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,above board,"честный (игроки показывали свою честность, дер...",True,False
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True,False
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас",True,False
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег",True,False
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...,False,False
5,en-ru,call a spade a spade,называть вещи своими именами,True,True
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую...",True,False
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...,True,False
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось",True,False
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)",True,False


In [202]:
for_the_web(draft_df)

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,above board,"честный (игроки показывали свою честность, дер...",True,False
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True,False
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас",True,False
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег",True,False
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...,False,False
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую...",True,False
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...,True,False
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось",True,False
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)",True,False
10,en-ru,follow suit,"иметь карту такой же масти, поступать также к...",True,False


In [205]:
draft_df = draft_df.drop(columns = ['check_phrase', 'check_transl'],axis = 1)
draft_df

,направление,фраза,перевод
0,en-ru,above board,"честный (игроки показывали свою честность, дер..."
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас"
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег"
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...
5,en-ru,call a spade a spade,называть вещи своими именами
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую..."
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось"
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)"


In [207]:
create_the_app_df(draft_df)
#create_the_web_df(df)

,направление,фраза,перевод,check_phrase,check_transl
5,en-ru,call a spade a spade,называть вещи своими именами,True,True
13,en-ru,have an ace up one's sleeve,"иметь козырь ""про запас""",True,True
18,en-ru,in the cards,"вероятный, возможный",True,True
30,en-ru,up the ante,повысить ставки,True,True


In [209]:
create_the_web_df(draft_df)

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,above board,"честный (игроки показывали свою честность, дер...",True,False
1,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True,False
2,en-ru,ace in the hole,"скрытое преимущество, козырь про запас",True,False
3,en-ru,ante up,"делать взнос, платить необходимую сумму денег",True,False
4,en-ru,be dealt a bad hand/deal someone a bad hand,получить плохие карты или попасть в невыгодное...,False,False
6,en-ru,call someone's bluff,"разоблачить обман, вывести кого-либо на чистую...",True,False
7,en-ru,cash in one's chips,умереть (дословно 'поменять фишки на наличные ...,True,False
8,en-ru,come/turn up trumps,"оказаться более удачным, чем предполагалось",True,False
9,en-ru,deal someone in,"включить кого-либо (в команду, в группу и т.д.)",True,False
10,en-ru,follow suit,"иметь карту такой же масти, поступать также к...",True,False


## Collecting idioms for the website and the application

In [210]:
list(tables_dict.keys())

['animals_en_ru_idioms_df',
 'birds_en_ru_idioms_df',
 'body_en_ru_idioms_df',
 'business_en_ru_idioms_df',
 'cards_en_ru_idioms_df',
 'clothes_en_ru_idioms_df',
 'color_en_ru_idioms_df',
 'education_en_ru_idioms_df',
 'eyes_and_ears_en_ru_idioms_df',
 'food_en_ru_idioms_df',
 'hand_and_arm_en_ru_idioms_df',
 'head_and_mind_en_ru_idioms_df',
 'heart_en_ru_idioms_df',
 'insects_and_fish_en_ru_idioms_df',
 'law_en_ru_idioms_df',
 'leg_and_foot_en_ru_idioms_df',
 'medical_en_ru_idioms_df',
 'money_en_ru_idioms_df',
 'mouth_and_teeth_en_ru_idioms_df',
 'negotiations_en_ru_idioms_df',
 'numbers_en_ru_idioms_df',
 'relationships_en_ru_idioms_df',
 'sports_en_ru_idioms_df']

In [212]:
for_the_app_tables_dict = {}


for name in list(tables_dict.keys()):
    current_table = tables_dict[name]
    for_the_app_table_name = 'for_the_app_' + name
    for_the_app_tables_dict[for_the_app_table_name] = create_the_app_df(current_table)
    
    
len(list(for_the_app_tables_dict.keys()))

23

In [213]:
list(for_the_app_tables_dict.keys())

['for_the_app_animals_en_ru_idioms_df',
 'for_the_app_birds_en_ru_idioms_df',
 'for_the_app_body_en_ru_idioms_df',
 'for_the_app_business_en_ru_idioms_df',
 'for_the_app_cards_en_ru_idioms_df',
 'for_the_app_clothes_en_ru_idioms_df',
 'for_the_app_color_en_ru_idioms_df',
 'for_the_app_education_en_ru_idioms_df',
 'for_the_app_eyes_and_ears_en_ru_idioms_df',
 'for_the_app_food_en_ru_idioms_df',
 'for_the_app_hand_and_arm_en_ru_idioms_df',
 'for_the_app_head_and_mind_en_ru_idioms_df',
 'for_the_app_heart_en_ru_idioms_df',
 'for_the_app_insects_and_fish_en_ru_idioms_df',
 'for_the_app_law_en_ru_idioms_df',
 'for_the_app_leg_and_foot_en_ru_idioms_df',
 'for_the_app_medical_en_ru_idioms_df',
 'for_the_app_money_en_ru_idioms_df',
 'for_the_app_mouth_and_teeth_en_ru_idioms_df',
 'for_the_app_negotiations_en_ru_idioms_df',
 'for_the_app_numbers_en_ru_idioms_df',
 'for_the_app_relationships_en_ru_idioms_df',
 'for_the_app_sports_en_ru_idioms_df']

In [214]:
sum_of_lenghts = 0
for table_name in list(for_the_app_tables_dict.keys()):
    length = len(for_the_app_tables_dict[table_name])
    sum_of_lenghts += length
print(sum_of_lenghts)

895


In [215]:
for_the_web_tables_dict = {}


for name in list(tables_dict.keys()):
    current_table = tables_dict[name]
    for_the_web_table_name = 'for_the_web_' + name
    for_the_web_tables_dict[for_the_web_table_name] = create_the_web_df(current_table)
    
    
len(list(for_the_web_tables_dict.keys()))

23

In [216]:
sum_of_lenghts = 0
for table_name in list(for_the_web_tables_dict.keys()):
    length = len(for_the_web_tables_dict[table_name])
    sum_of_lenghts += length
print(sum_of_lenghts)

1618


In [271]:
list(for_the_web_tables_dict.keys())

['for_the_web_animals_en_ru_idioms_df',
 'for_the_web_birds_en_ru_idioms_df',
 'for_the_web_body_en_ru_idioms_df',
 'for_the_web_business_en_ru_idioms_df',
 'for_the_web_cards_en_ru_idioms_df',
 'for_the_web_clothes_en_ru_idioms_df',
 'for_the_web_color_en_ru_idioms_df',
 'for_the_web_education_en_ru_idioms_df',
 'for_the_web_eyes_and_ears_en_ru_idioms_df',
 'for_the_web_food_en_ru_idioms_df',
 'for_the_web_hand_and_arm_en_ru_idioms_df',
 'for_the_web_head_and_mind_en_ru_idioms_df',
 'for_the_web_heart_en_ru_idioms_df',
 'for_the_web_insects_and_fish_en_ru_idioms_df',
 'for_the_web_law_en_ru_idioms_df',
 'for_the_web_leg_and_foot_en_ru_idioms_df',
 'for_the_web_medical_en_ru_idioms_df',
 'for_the_web_money_en_ru_idioms_df',
 'for_the_web_mouth_and_teeth_en_ru_idioms_df',
 'for_the_web_negotiations_en_ru_idioms_df',
 'for_the_web_numbers_en_ru_idioms_df',
 'for_the_web_relationships_en_ru_idioms_df',
 'for_the_web_sports_en_ru_idioms_df']

# Another source

In [217]:
with open("/Users/dmitrijsuhotin/Desktop/idioms en-ru/idioms#1.html", "r") as f: #url: https://skysmart.ru/articles/english/idiomy-na-anglijskom-yazyke
    contents = f.read()
    soup = bs(contents, 'lxml')

In [218]:
idioms = []
translations = []
for tag in soup.find_all("p")[20:95]:
    s = tag.text
    if len(s.split('—')) > 1:
        phrase = s.split('—')[0][:-1]
        mean = s.split('—')[1][1:-1]
        idioms.append(phrase)
        translations.append(mean)

#idioms[:5]
#translations[:5]
#print("{0}: {1}".format(tag.name, tag.text))

    
direct = ['en-ru'] * 72
en_to_ru_idioms = {'направление': direct, 'фраза': idioms, 'перевод': translations}
en_to_ru_df = pd.DataFrame(en_to_ru_idioms)
en_to_ru_df

,направление,фраза,перевод
0,en-ru,As cool as a cucumber,"невозмутимый человек, у которого все под контр..."
1,en-ru,To be in a pickle,"оказаться в трудной ситуации, как маринованные..."
2,en-ru,Couch Potato,"лежебока, лентяй, тот, кто предпочитает диван ..."
3,en-ru,Like a sack of potatoes,"сравнение с мешком картошки предполагает, что ..."
4,en-ru,Two peas in a pod,"то же, что и русское «как две капли воды», тол..."
...,...,...,...
67,en-ru,To lose your touch,потерять способности к чему-то; потерять сноровку
68,en-ru,To be on the fence,трудность в принятии решения; быть в замешател...
69,en-ru,To kick the bucket,"умереть, сыграть в ящик, склеить ласты, отброс..."
70,en-ru,To show someone the ropes,"показать, как нужно выполнять ту или иную рабо..."


In [219]:
for_app_different_en_ru_idioms_df = create_the_app_df(en_to_ru_df)
for_app_different_en_ru_idioms_df.reset_index()

,index,направление,фраза,перевод,check_phrase,check_transl
0,5,en-ru,To be paid peanuts,"работать за копейки, арахис",True,True
1,6,en-ru,To go nuts,"обезуметь, сойти с ума",True,True
2,16,en-ru,"Monkey see, monkey do","обезьянничать, подражать",True,True
3,26,en-ru,Pretty penny,"крупная, значительная сумма",True,True
4,29,en-ru,Money for old rope,легкие деньги,True,True
5,31,en-ru,To blow your own trumpet,нахваливать себя,True,True
6,35,en-ru,Call the tune,"контролировать, распоряжаться",True,True
7,40,en-ru,To play second fiddle,быть на вторых ролях,True,True
8,41,en-ru,To have a whale of a time,прекрасно провести время,True,True
9,42,en-ru,Pressed for time,"быть занятым, сильно спешить",True,True


In [221]:
different_phrases = list(for_app_different_en_ru_idioms_df['фраза'])
len(different_phrases)

22

In [224]:
list_of_all_app_idioms = []
for table_name in list(for_the_app_tables_dict.keys()):
    idioms_table = for_the_app_tables_dict[table_name]
    list_of_all_app_idioms += list(idioms_table['фраза'])
    
print(len(list_of_all_app_idioms))


895


In [229]:
unique_different_phrases = []
for el in for_app_different_en_ru_idioms_df['фраза']:
    if el not in list_of_all_app_idioms:
        unique_different_phrases.append(el)
unique_different_phrases

['To be paid peanuts',
 'To go nuts',
 'Monkey see, monkey do',
 'Pretty penny',
 'Money for old rope',
 'To blow your own trumpet',
 'Call the tune',
 'To play second fiddle',
 'To have a whale of a time',
 'Pressed for time',
 'A waste of time',
 'To run out of time/time is up',
 'Behind the times',
 'To take one’s time',
 'Time is money',
 'It never rains but it pours',
 'Under the weather',
 'As right as rain',
 'Chase rainbows',
 'Have your head in the clouds',
 'To twist someone’s arm',
 'To be up in the air']

In [230]:
for_the_app_tables_dict['for_the_app_different_en_ru_idioms_df'] = for_app_different_en_ru_idioms_df

In [231]:
list(for_the_app_tables_dict.keys())

['for_the_app_animals_en_ru_idioms_df',
 'for_the_app_birds_en_ru_idioms_df',
 'for_the_app_body_en_ru_idioms_df',
 'for_the_app_business_en_ru_idioms_df',
 'for_the_app_cards_en_ru_idioms_df',
 'for_the_app_clothes_en_ru_idioms_df',
 'for_the_app_color_en_ru_idioms_df',
 'for_the_app_education_en_ru_idioms_df',
 'for_the_app_eyes_and_ears_en_ru_idioms_df',
 'for_the_app_food_en_ru_idioms_df',
 'for_the_app_hand_and_arm_en_ru_idioms_df',
 'for_the_app_head_and_mind_en_ru_idioms_df',
 'for_the_app_heart_en_ru_idioms_df',
 'for_the_app_insects_and_fish_en_ru_idioms_df',
 'for_the_app_law_en_ru_idioms_df',
 'for_the_app_leg_and_foot_en_ru_idioms_df',
 'for_the_app_medical_en_ru_idioms_df',
 'for_the_app_money_en_ru_idioms_df',
 'for_the_app_mouth_and_teeth_en_ru_idioms_df',
 'for_the_app_negotiations_en_ru_idioms_df',
 'for_the_app_numbers_en_ru_idioms_df',
 'for_the_app_relationships_en_ru_idioms_df',
 'for_the_app_sports_en_ru_idioms_df',
 'for_the_app_different_en_ru_idioms_df']

# One more source

In [248]:
url = 'http://www.correctenglish.ru/reference/idioms/c/'
r = requests.get(url)
html = bs(r.content, 'html.parser')
items = html.select(".long_list > a")

idioms = []
for el in items:
    idioms.append(el.text)
#len(idioms)

In [249]:
items1 = html.select(".long_list > .ref")
meanings = []
for el in items1:
    meanings.append(el.text)
    
len(meanings)

105

In [239]:
list(zip(idioms, meanings))

[("(not) one's cup of tea", '(не) то, что нравится, (не) быть по вкусу'),
 ('a case in point',
  'пример, который доказывает что-либо в текущей ситуации, или который помогает прояснить что-либо'),
 ('a clean bill of health',
  '1. свидетельство, удостоверяющее, что данный человек здоров; подтверждение того, что кто-либо здоров2. гарантия качества, доказательство отсутствия неисправностей, правильной, хорошей работы чего-либо (какого-либо устройства, организации, и т.п.)'),
 ('a copycat',
  'подражатель; кто-либо, кто подражает, имитирует действия, поступки других'),
 ('a course of action',
  'ход действий; что-либо, чего нужно придерживаться, чтобы что-либо сделать'),
 ('call a spade a spade', 'называть вещи своими именами'),
 ('call it a day / call it a night',
  'считать что-либо законченным; закончить работу, и пойти домой'),
 ('call it quits', 'прекратить, покончить, поставить точку'),
 ('call someone or something into question',
  'подвергать сомнению, усомниться'),
 ('call the sh

In [241]:
list(r'[A-Z]')

['[', 'A', '-', 'Z', ']']

In [244]:
import string
for l in string.ascii_lowercase:
    print(l)

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


## The entire website parsing

In [250]:
#url = 'http://www.correctenglish.ru/reference/idioms/'
alphabet = string.ascii_lowercase[1:]
for l in alphabet:
    url = 'http://www.correctenglish.ru/reference/idioms/' + l + '/'
    r = requests.get(url)
    html = bs(r.content, 'html.parser')
    
    
    items = html.select(".long_list > a")
    #idioms = []
    for el in items:
        idioms.append(el.text)
        
    
    items1 = html.select(".long_list > .ref")
    #meanings = []
    for el in items1:
        meanings.append(el.text)

        
print(len(idioms), len(meanings))

1974 1974


In [251]:
direct = ['en-ru'] * 1974
alph_en_to_ru_idioms = {'направление': direct, 'фраза': idioms, 'перевод': meanings}
alph_en_to_ru_df = pd.DataFrame(alph_en_to_ru_idioms)
alph_en_to_ru_df

,направление,фраза,перевод
0,en-ru,(not) one's cup of tea,"(не) то, что нравится, (не) быть по вкусу"
1,en-ru,a case in point,"пример, который доказывает что-либо в текущей ..."
2,en-ru,a clean bill of health,"1. свидетельство, удостоверяющее, что данный ч..."
3,en-ru,a copycat,"подражатель; кто-либо, кто подражает, имитируе..."
4,en-ru,a course of action,"ход действий; что-либо, чего нужно придерживат..."
...,...,...,...
1969,en-ru,Your secret is safe with me.,я никому не расскажу твою тайну
1970,en-ru,yowza / yowzah,"восклицание, выражающее удивление, изумление, ..."
1971,en-ru,zero hour,"точное время, в которое начнется нападение или..."
1972,en-ru,zero-sum game,"ситуация, в которой если кто-либо выигрывает и..."


In [252]:
for_app_alph_en_ru_idioms_df = create_the_app_df(alph_en_to_ru_df)
for_app_alph_en_ru_idioms_df.reset_index()

,index,направление,фраза,перевод,check_phrase,check_transl
0,5,en-ru,call a spade a spade,называть вещи своими именами,True,True
1,23,en-ru,cast one's vote,голосовать,True,True
2,24,en-ru,castles in the air,"воздушные замки, мечты",True,True
3,26,en-ru,catch a cold,простудиться,True,True
4,28,en-ru,catch someone off guard,застать кого-либо врасплох,True,True
...,...,...,...,...,...,...
468,1955,en-ru,yakety-yak,много разговоров по пустякам,True,True
469,1956,en-ru,year after year,"много лет, год за годом",True,True
470,1959,en-ru,yellow streak,трусость,True,True
471,1960,en-ru,yellow-bellied,трусливый,True,True


In [253]:
alph_phrases = list(for_app_alph_en_ru_idioms_df['фраза'])
len(alph_phrases)

473

In [254]:
list_of_all_app_idioms = []
for table_name in list(for_the_app_tables_dict.keys()):
    idioms_table = for_the_app_tables_dict[table_name]
    list_of_all_app_idioms += list(idioms_table['фраза'])
    
print(len(list_of_all_app_idioms))

917


In [255]:
unique_alph_phrases = []
for el in for_app_alph_en_ru_idioms_df['фраза']:
    if el not in list_of_all_app_idioms:
        unique_alph_phrases.append(el)
len(unique_alph_phrases)

406

In [259]:
for_app_alph_en_ru_idioms_df = for_app_alph_en_ru_idioms_df[for_app_alph_en_ru_idioms_df['фраза'].isin(unique_alph_phrases)]
for_app_alph_en_ru_idioms_df

,направление,фраза,перевод,check_phrase,check_transl
23,en-ru,cast one's vote,голосовать,True,True
24,en-ru,castles in the air,"воздушные замки, мечты",True,True
28,en-ru,catch someone off guard,застать кого-либо врасплох,True,True
36,en-ru,change the subject,сменить тему разговора,True,True
38,en-ru,cheat sheet,шпаргалка,True,True
...,...,...,...,...,...
1955,en-ru,yakety-yak,много разговоров по пустякам,True,True
1956,en-ru,year after year,"много лет, год за годом",True,True
1959,en-ru,yellow streak,трусость,True,True
1960,en-ru,yellow-bellied,трусливый,True,True


In [260]:
list(for_app_alph_en_ru_idioms_df['фраза']) == unique_alph_phrases

True

In [261]:
for_the_app_tables_dict['for_app_alph_en_ru_idioms_df'] = for_app_alph_en_ru_idioms_df

In [262]:
list(for_the_app_tables_dict.keys())

['for_the_app_animals_en_ru_idioms_df',
 'for_the_app_birds_en_ru_idioms_df',
 'for_the_app_body_en_ru_idioms_df',
 'for_the_app_business_en_ru_idioms_df',
 'for_the_app_cards_en_ru_idioms_df',
 'for_the_app_clothes_en_ru_idioms_df',
 'for_the_app_color_en_ru_idioms_df',
 'for_the_app_education_en_ru_idioms_df',
 'for_the_app_eyes_and_ears_en_ru_idioms_df',
 'for_the_app_food_en_ru_idioms_df',
 'for_the_app_hand_and_arm_en_ru_idioms_df',
 'for_the_app_head_and_mind_en_ru_idioms_df',
 'for_the_app_heart_en_ru_idioms_df',
 'for_the_app_insects_and_fish_en_ru_idioms_df',
 'for_the_app_law_en_ru_idioms_df',
 'for_the_app_leg_and_foot_en_ru_idioms_df',
 'for_the_app_medical_en_ru_idioms_df',
 'for_the_app_money_en_ru_idioms_df',
 'for_the_app_mouth_and_teeth_en_ru_idioms_df',
 'for_the_app_negotiations_en_ru_idioms_df',
 'for_the_app_numbers_en_ru_idioms_df',
 'for_the_app_relationships_en_ru_idioms_df',
 'for_the_app_sports_en_ru_idioms_df',
 'for_the_app_different_en_ru_idioms_df',
 'for_

## Saving the data

In [264]:
import pickle

In [265]:
with open('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/for_the_app_tables_dict', 'wb') as f:
    pickle.dump(for_the_app_tables_dict, f)

In [279]:
with open('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/for_the_app_tables_dict', 'rb') as f:
    my_fav_dict = pickle.load(f)

In [280]:
list(my_fav_dict.keys())

['for_the_app_animals_en_ru_idioms_df',
 'for_the_app_birds_en_ru_idioms_df',
 'for_the_app_body_en_ru_idioms_df',
 'for_the_app_business_en_ru_idioms_df',
 'for_the_app_cards_en_ru_idioms_df',
 'for_the_app_clothes_en_ru_idioms_df',
 'for_the_app_color_en_ru_idioms_df',
 'for_the_app_education_en_ru_idioms_df',
 'for_the_app_eyes_and_ears_en_ru_idioms_df',
 'for_the_app_food_en_ru_idioms_df',
 'for_the_app_hand_and_arm_en_ru_idioms_df',
 'for_the_app_head_and_mind_en_ru_idioms_df',
 'for_the_app_heart_en_ru_idioms_df',
 'for_the_app_insects_and_fish_en_ru_idioms_df',
 'for_the_app_law_en_ru_idioms_df',
 'for_the_app_leg_and_foot_en_ru_idioms_df',
 'for_the_app_medical_en_ru_idioms_df',
 'for_the_app_money_en_ru_idioms_df',
 'for_the_app_mouth_and_teeth_en_ru_idioms_df',
 'for_the_app_negotiations_en_ru_idioms_df',
 'for_the_app_numbers_en_ru_idioms_df',
 'for_the_app_relationships_en_ru_idioms_df',
 'for_the_app_sports_en_ru_idioms_df',
 'for_the_app_different_en_ru_idioms_df',
 'for_

In [268]:
my_fav_dict['for_the_app_different_en_ru_idioms_df']

,направление,фраза,перевод,check_phrase,check_transl
5,en-ru,To be paid peanuts,"работать за копейки, арахис",True,True
6,en-ru,To go nuts,"обезуметь, сойти с ума",True,True
16,en-ru,"Monkey see, monkey do","обезьянничать, подражать",True,True
26,en-ru,Pretty penny,"крупная, значительная сумма",True,True
29,en-ru,Money for old rope,легкие деньги,True,True
31,en-ru,To blow your own trumpet,нахваливать себя,True,True
35,en-ru,Call the tune,"контролировать, распоряжаться",True,True
40,en-ru,To play second fiddle,быть на вторых ролях,True,True
41,en-ru,To have a whale of a time,прекрасно провести время,True,True
42,en-ru,Pressed for time,"быть занятым, сильно спешить",True,True


In [269]:
list_of_all_app_idioms = []
for table_name in list(my_fav_dict.keys()):
    idioms_table = my_fav_dict[table_name]
    list_of_all_app_idioms += list(idioms_table['фраза'])
    
print(len(list_of_all_app_idioms))

1323


In [272]:
with open('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/for_the_web_tables_dict', 'wb') as f: # тут надо написать файловый путь самостоятельно
    pickle.dump(for_the_web_tables_dict, f) # всего: 1618 идиом

In [281]:
with open('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/for_the_web_tables_dict', 'rb') as f:
    my_fav_dict1 = pickle.load(f)

In [282]:
list_of_all_web_idioms = []
for table_name in list(my_fav_dict1.keys()):
    idioms_table = my_fav_dict1[table_name]
    list_of_all_web_idioms += list(idioms_table['фраза'])
    

print(len(list_of_all_web_idioms))

1618


## Concatenation and dataframe sorting

In [363]:
def aph_sort(df: pd.DataFrame, col: str): # вводим df и колонку, по которой выполняется сортировка 
    lower_col_name = 'lower_' + col
    df[lower_col_name] = [el.lower() for el in list(df[col])]
    
    
    
    df.sort_values(by = lower_col_name, inplace = True)
    df.drop(columns = [lower_col_name], axis = 1, inplace = True)
    return df.reset_index().drop(columns = ['index'], axis = 1)



dict1 = {'Ann': [1, 10, 23], 'Bob': [12, 92, 46]}
dict2 = {'Ann': [101, 263], 'Bob': [62, 467]}


df1 = pd.DataFrame(dict1)
df2 = pd.DataFrame(dict2)

all_grades = pd.concat([df1, df2])


all_grades['needs'] = ['pen', 'apple', 'alphabet', 'English book', 'database']
# all_grades['lower_needs'] = [el.lower() for el in list(all_grades['needs'])]


# all_grades.sort_values(by = 'lower_needs', inplace = True)
# all_grades.drop(columns = ['lower_needs'], axis = 1, inplace = True)
# all_grades


aph_sort(all_grades, 'needs')

#all_grades

,Ann,Bob,needs
0,23,46,alphabet
1,10,92,apple
2,263,467,database
3,101,62,English book
4,1,12,pen


In [344]:
needs = all_grades['needs'] #['pen', 'apple', 'alphabet', 'English book', 'database']
needs = sorted(needs, key = lambda need: need.lower())
print(needs)

['alphabet', 'apple', 'database', 'English book', 'pen']


In [345]:
all_grades['needs']

0             pen
1           apple
2        alphabet
0    English book
1        database
Name: needs, dtype: object

In [318]:
all_grades.reset_index()

,index,Ann,Bob,needs
0,2,23,46,alphabet
1,1,10,92,apple
2,1,263,467,database
3,0,101,62,english book
4,0,1,12,pen


In [309]:
print(list(all_grades['needs']).sort())

None


NameError: name 'Series' is not defined

In [315]:
print(sorted(a))

['English book', 'alphabet', 'apple', 'database', 'pen']


## Concatenation

In [293]:
all_for_app_df = my_fav_dict[list(my_fav_dict.keys())[0]]


for k in list(my_fav_dict.keys())[1:]:
    curr_df = my_fav_dict[k]
    #print(len(curr_df))
    all_for_app_df = pd.concat([all_for_app_df, curr_df])

    
print(len(all_for_app_df))

#all_for_app_df

1323


In [294]:
all_for_web_df = my_fav_dict1[list(my_fav_dict1.keys())[0]]


for k in list(my_fav_dict1.keys())[1:]:
    curr_df = my_fav_dict1[k]
    #print(len(curr_df))
    all_for_web_df = pd.concat([all_for_web_df, curr_df])

    
print(len(all_for_web_df))

1618


In [316]:
all_for_app_df[:100]

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,ahead of the pack,(быть) впереди всех,True,True
1,en-ru,alley cat,"бродячая, бездомная кошка",True,True
3,en-ru,as blind as a bat,"слепой, ""слепой как крот""",True,True
4,en-ru,as busy as a beaver,быть очень занятым,True,True
5,en-ru,as clean as a hound's tooth,очень чистый,True,True
...,...,...,...,...,...
26,en-ru,by the sweat of one's brow,работать в поте лица,True,True
28,en-ru,carry one's (own) weight,делать свою долю работы,True,True
30,en-ru,catch one's breath,"перевести дух, передохнуть",True,True
31,en-ru,chilled to the bone,промерзнуть до костей,True,True


## Alphabet sorting

In [365]:
all_for_app_df = aph_sort(all_for_app_df, 'фраза')
all_for_app_df

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,(as) old as the hills,очень старый,True,True
1,en-ru,(to be) half the battle,большая часть работы,True,True
2,en-ru,(to be) hand in hand,держаться за руки,True,True
3,en-ru,a doubting Thomas,Фома неверующий,True,True
4,en-ru,a feather in one's cap,предмет особой гордости,True,True
...,...,...,...,...,...
1318,en-ru,yellow streak,трусость,True,True
1319,en-ru,yellow-bellied,трусливый,True,True
1320,en-ru,you can't please everyone,всем не угодишь,True,True
1321,en-ru,young at heart,(быть) молодым душой,True,True


In [366]:
all_for_web_df = aph_sort(all_for_web_df, 'фраза')
all_for_web_df

,направление,фраза,перевод,check_phrase,check_transl
0,en-ru,able to (do something) standing on one's head,уметь делать что-либо легко и быстро,False,False
1,en-ru,able to (do something) with one's eyes closed/...,уметь очень легко делать что-либо (с закрытыми...,False,False
2,en-ru,able to breathe easily/freely again,снова вздохнуть свободно,False,True
3,en-ru,above board,"честный (игроки показывали свою честность, дер...",True,False
4,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ...",True,False
...,...,...,...,...,...
1613,en-ru,you can lead a horse to water (but you can't m...,"можно предоставить кому-либо возможность, но н...",False,False
1614,en-ru,you can't make an omelet without breaking the ...,"нельзя добиться чего-либо без труда, лес рубят...",False,False
1615,en-ru,you can't teach an old dog new tricks,в старости поздно учиться или переучиваться,False,False
1616,en-ru,You scratch my back and I'll scratch yours.,"обоюдные услуги, ""рука руку моет""",False,False


## Deleting redundant columns

In [369]:
all_for_app_df = all_for_app_df.drop(columns = ['check_phrase', 'check_transl'], axis = 1)
all_for_app_df

,направление,фраза,перевод
0,en-ru,(as) old as the hills,очень старый
1,en-ru,(to be) half the battle,большая часть работы
2,en-ru,(to be) hand in hand,держаться за руки
3,en-ru,a doubting Thomas,Фома неверующий
4,en-ru,a feather in one's cap,предмет особой гордости
...,...,...,...
1318,en-ru,yellow streak,трусость
1319,en-ru,yellow-bellied,трусливый
1320,en-ru,you can't please everyone,всем не угодишь
1321,en-ru,young at heart,(быть) молодым душой


In [370]:
all_for_web_df = all_for_web_df.drop(columns = ['check_phrase', 'check_transl'], axis = 1)
all_for_web_df

,направление,фраза,перевод
0,en-ru,able to (do something) standing on one's head,уметь делать что-либо легко и быстро
1,en-ru,able to (do something) with one's eyes closed/...,уметь очень легко делать что-либо (с закрытыми...
2,en-ru,able to breathe easily/freely again,снова вздохнуть свободно
3,en-ru,above board,"честный (игроки показывали свою честность, дер..."
4,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
...,...,...,...
1613,en-ru,you can lead a horse to water (but you can't m...,"можно предоставить кому-либо возможность, но н..."
1614,en-ru,you can't make an omelet without breaking the ...,"нельзя добиться чего-либо без труда, лес рубят..."
1615,en-ru,you can't teach an old dog new tricks,в старости поздно учиться или переучиваться
1616,en-ru,You scratch my back and I'll scratch yours.,"обоюдные услуги, ""рука руку моет"""


## Saving dataframes as csv

In [371]:
all_for_app_df.to_csv('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/all_for_app_df.csv') 
all_for_web_df.to_csv('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/all_for_web_df.csv') 

In [372]:
df = pd.read_csv('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/all_for_app_df.csv')
df

,Unnamed: 0,направление,фраза,перевод
0,0,en-ru,(as) old as the hills,очень старый
1,1,en-ru,(to be) half the battle,большая часть работы
2,2,en-ru,(to be) hand in hand,держаться за руки
3,3,en-ru,a doubting Thomas,Фома неверующий
4,4,en-ru,a feather in one's cap,предмет особой гордости
...,...,...,...,...
1318,1318,en-ru,yellow streak,трусость
1319,1319,en-ru,yellow-bellied,трусливый
1320,1320,en-ru,you can't please everyone,всем не угодишь
1321,1321,en-ru,young at heart,(быть) молодым душой


In [373]:
df1 = pd.read_csv('/Users/dmitrijsuhotin/Desktop/idioms en-ru/таблицы с идиомами/all_for_web_df.csv')
df1

,Unnamed: 0,направление,фраза,перевод
0,0,en-ru,able to (do something) standing on one's head,уметь делать что-либо легко и быстро
1,1,en-ru,able to (do something) with one's eyes closed/...,уметь очень легко делать что-либо (с закрытыми...
2,2,en-ru,able to breathe easily/freely again,снова вздохнуть свободно
3,3,en-ru,above board,"честный (игроки показывали свою честность, дер..."
4,4,en-ru,according to Hoyle,"в соответствии с правилами, по правилам (Хойл ..."
...,...,...,...,...
1613,1613,en-ru,you can lead a horse to water (but you can't m...,"можно предоставить кому-либо возможность, но н..."
1614,1614,en-ru,you can't make an omelet without breaking the ...,"нельзя добиться чего-либо без труда, лес рубят..."
1615,1615,en-ru,you can't teach an old dog new tricks,в старости поздно учиться или переучиваться
1616,1616,en-ru,You scratch my back and I'll scratch yours.,"обоюдные услуги, ""рука руку моет"""
